# <center>Séries temporais</center>
___

Um dos grandes desafios das empresas é a previsão de vendas. Um bom modelo de previsão das vendas da empresa permite um melhor planejamento de gastos das empresas e da produção, permite uma estimação de lucros e até mesmo auxilia a empresa a determinar metas, avaliar o seu desempenho e ter uma melhor visão de futuro, ajudando na atração de possíveis investidores.

Esse notebook contém um estudo prático sobre esse tema. O *dataset* que utilizaremos vem originalmente de um desafio de recrutamento do Walmart - que pode ser acessado [aqui](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data). Ele contém vendas anônimas por departamento para 45 lojas Walmart, bem como variáveis de apoio. Cada loja contém um número de departamentos, além disso, a Walmart realiza vários eventos promocionais de descontos (ou `Markdown`) ao longo do ano.

Assim, o objetivo proposto aqui é de **prever as vendas semanais de cada departamento de cada loja da Walmart**.


Vamos começar lendo o nosso *dataset*

In [ ]:
# Importando as bibliotecas principais utilizadas
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

# Importando o dataset
walmart = pd.read_csv("walmart_sales.csv")

# Mostrando o formato e as primeiras linhas do dataset
print(walmart.shape)
walmart.head()

(282451, 16)


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,26,92,2011-08-26,87235.57,False,61.10,3.796,NaN,NaN,NaN,NaN,NaN,136.213613,7.767,A,152513
1,34,22,2011-03-25,5945.97,False,53.11,3.480,NaN,NaN,NaN,NaN,NaN,128.616064,10.398,A,158114
2,21,28,2010-12-03,1219.89,False,50.43,2.708,NaN,NaN,NaN,NaN,NaN,211.265543,8.163,B,140167
3,8,9,2010-09-17,11972.71,False,75.32,2.582,NaN,NaN,NaN,NaN,NaN,214.878556,6.315,A,155078
4,19,55,2012-05-18,8271.82,False,58.81,4.029,12613.98,NaN,11.5,1705.28,3600.79,138.106581,8.150,A,203819


Já vimos que existem alguns valores nulos. Vamos utilizar os métodos `info` e `describe` para identificar com maior detalhe algumas informações necessárias para fazer uma limpeza básica nos nossos dados.

In [ ]:
walmart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282451 entries, 0 to 282450
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         282451 non-null  int64  
 1   Dept          282451 non-null  int64  
 2   Date          282451 non-null  object 
 3   Weekly_Sales  282451 non-null  float64
 4   IsHoliday     282451 non-null  bool   
 5   Temperature   282451 non-null  float64
 6   Fuel_Price    282451 non-null  float64
 7   MarkDown1     100520 non-null  float64
 8   MarkDown2     74232 non-null   float64
 9   MarkDown3     91521 non-null   float64
 10  MarkDown4     90031 non-null   float64
 11  MarkDown5     101029 non-null  float64
 12  CPI           282451 non-null  float64
 13  Unemployment  282451 non-null  float64
 14  Type          282451 non-null  object 
 15  Size          282451 non-null  int64  
dtypes: bool(1), float64(10), int64(3), object(2)
memory usage: 32.6+ MB


In [ ]:
walmart.describe(include='all')

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
count,282451.000000,282451.000000,282451,282451.000000,282451,282451.000000,282451.000000,100520.000000,74232.000000,91521.000000,90031.000000,101029.000000,282451.000000,282451.000000,282451,282451.000000
unique,NaN,NaN,143,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN
top,NaN,NaN,2010-09-24,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN
freq,NaN,NaN,2044,NaN,262632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144559,NaN
mean,22.193166,44.286138,NaN,15983.429692,NaN,60.113640,3.360300,7246.077559,3318.408122,1417.397841,3379.591745,4639.476021,171.207802,7.968098,NaN,136730.073220
std,12.782138,30.503641,NaN,22661.092494,NaN,18.446485,0.458602,8254.606267,9485.575898,9547.858949,6269.428446,6060.459590,39.160808,1.868070,NaN,61002.319363
min,1.000000,1.000000,NaN,-4988.940000,NaN,-2.060000,2.472000,0.270000,-265.760000,-29.100000,0.220000,135.160000,126.064000,3.879000,NaN,34875.000000
25%,11.000000,18.000000,NaN,2079.330000,NaN,46.780000,2.932000,2241.190000,40.960000,5.060000,508.100000,1877.810000,132.022667,6.891000,NaN,93638.000000
50%,22.000000,38.000000,NaN,7616.550000,NaN,62.150000,3.452000,5363.520000,191.820000,24.340000,1482.030000,3364.410000,182.350989,7.866000,NaN,140167.000000
75%,33.000000,74.000000,NaN,20245.745000,NaN,74.290000,3.737000,9235.590000,1919.790000,103.130000,3607.570000,5563.800000,212.464799,8.572000,NaN,202505.000000


Por meio de uma verificação visual, já sabemos que temos valores nulos, mas vamos confirmar isso através delinhas de código:

In [ ]:
walmart.isna().any()

Store           False
Dept            False
Date            False
Weekly_Sales    False
IsHoliday       False
Temperature     False
Fuel_Price      False
MarkDown1        True
MarkDown2        True
MarkDown3        True
MarkDown4        True
MarkDown5        True
CPI             False
Unemployment    False
Type            False
Size            False
dtype: bool

Dessa rápida análise, conseguimos tirar alguns insights para o tratamento dos nossos dados:

1. A coluna 'Date' precisa ser transformada para o formato 'DateTime' para trabalharmos melhor com ela
2. A coluna 'IsHoliday' é booleana e precisa ser transformada em numérica
3. A coluna 'Type' representa dados categóricos em formato de string. Para o input no modelo, essa coluna precisa ser transformada em Dummy
4. As colunas 'MarkDown' possuem uma significativa quantidade de dados vazios
5. As colunas 'Store' e 'Dept' são numéricas e discretas

Vamos iniciar transformando a variável `'Date'` no formato `'DateTime'` para facilitar a manipulação do *dataset*

In [ ]:
# Transforme a variavel 'Date' em DateTime
walmart['Date'] =  pd.to_datetime(walmart['Date'])

# Ordenando os valores do dataset inicialmente por 'Date', depois 'Store' e 'Dept'
walmart.sort_values(["Date","Store","Dept"], inplace=True)

# walmart_types = walmart.dtypes

print(walmart[['Date']].info())
walmart.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282451 entries, 201743 to 172694
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Date    282451 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 4.3 MB
None


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
201743,1,4,2010-02-05,39954.04,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
121,1,5,2010-02-05,32229.38,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
2692,1,8,2010-02-05,40129.01,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
82375,1,9,2010-02-05,16930.99,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
98189,1,10,2010-02-05,30721.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
185298,1,12,2010-02-05,8449.54,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
230439,1,13,2010-02-05,41969.29,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
193952,1,14,2010-02-05,19466.91,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
79712,1,16,2010-02-05,10217.55,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
47797,1,19,2010-02-05,1947.05,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315


Podemos ver que existem várias linhas com datas iguais. Isso significa que, para uma mesma data, existem várias linhas com dados diferentes, o que não é ideal quando queremos trabalhar com séries temporais, já que queremos que o tempo seja o índice do nosso *dataset*. Contudo, se você perceber do resultado do `.head` acima, os dados das colunas 'Store' e 'Dept' são diferentes para cada data. Vamos analisar se, para uma mesma data, loja e departamento, existem informações duplicadas.

In [ ]:
# Verificando o número de datas distintas
len(walmart.Date.unique())

143

In [ ]:
# Verificando se uma mesma data possui dados com store e dept iguais
walmart.duplicated(['Date','Store','Dept']).sum()

0

Perfeito! Notamos então que, no mesmo *dataset*, temos diferentes séries temporais, cada uma caracterizada por essas três colunas: 'Date', 'Store' e 'Dept'. Vamos visualizar a evolução das vendas ao longo do tempo para uma dada loja e departamento.

Obs.: A barra abaixo auxilia a explorar diferentes séries temporais.

In [ ]:
# Plotando a série temporal para uma dada loja e departamento

from IPython.display import display
from ipywidgets import interact, IntSlider


def plot_serie(store, dept):
    plot_filter = walmart[(walmart['Store']==store) & (walmart['Dept']==dept)]

    fig, ax = plt.subplots(figsize=(10,3))
    ax.plot(plot_filter['Date'],plot_filter['Weekly_Sales'])

    plt.show()

# Criando o Slider interativo
_ = interact(plot_serie,
             store=IntSlider(min=walmart.Store.min(), max=walmart.Store.max(), step = 1, value = 9),
             dept =IntSlider(min=walmart.Dept.min(), max=walmart.Dept.max(), step = 1, value = 71)
             )

interactive(children=(IntSlider(value=9, description='store', max=45, min=1), IntSlider(value=71, description=…

Agora vamos tratar os dados e criar uma feature.

Como vimos, os valores nulos são das colunas Markdown, que representam os descontos dados pelos departamentos, dessa forma, vamos substituir os valores nulos do dataset por zeros.

In [ ]:
walmart = walmart.fillna(0)

Em seguida, vamos transformar a variável 'IsHoliday'em int.

In [ ]:
walmart['IsHoliday'] = pd.to_numeric(walmart['IsHoliday']).astype('int')

Criaremos a feature 'days_from' que indica o intervalo de dias desde a data mínima (int).

In [ ]:
min_date = walmart['Date'].min()                             # Encontre a data mínima da coluna de datas
walmart['days_from'] = walmart['Date'] - min_date         # Calcule a diferença entre a data e a data mínima (formato timedelta)
walmart.days_from = walmart.days_from.dt.days      # Pegando apenas os "dias" como int

Agora criaremos Dummies para as variáveis categóricas do dataset e verificaremos as novas informações.

A função get_dummies no Pandas é usada para criar variáveis dummy (também conhecidas como variáveis fictícias ou variáveis indicadoras) a partir de uma coluna que contenha dados categóricos. Essa função é frequentemente usada em análise de dados e modelagem estatística, especialmente em tarefas de aprendizado de máquina.

Quando você tem uma coluna com valores categóricos, como "gato", "cachorro" e "peixe", o get_dummies cria colunas separadas para cada categoria e preenche essas colunas com 0s e 1s para indicar a presença ou ausência de cada categoria em cada observação de dados.

In [ ]:
walmart = pd.get_dummies(walmart)
walmart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282451 entries, 201743 to 172694
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         282451 non-null  int64         
 1   Dept          282451 non-null  int64         
 2   Date          282451 non-null  datetime64[ns]
 3   Weekly_Sales  282451 non-null  float64       
 4   IsHoliday     282451 non-null  int64         
 5   Temperature   282451 non-null  float64       
 6   Fuel_Price    282451 non-null  float64       
 7   MarkDown1     282451 non-null  float64       
 8   MarkDown2     282451 non-null  float64       
 9   MarkDown3     282451 non-null  float64       
 10  MarkDown4     282451 non-null  float64       
 11  MarkDown5     282451 non-null  float64       
 12  CPI           282451 non-null  float64       
 13  Unemployment  282451 non-null  float64       
 14  Size          282451 non-null  int64         
 15  Type_A      

Agora vamos realizar a separação do nosso *dataset* em conjunto de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split # Importa a função usada para a separação de treino e teste
from sklearn.model_selection import TimeSeriesSplit # Importa a função usada para a separação da validação cruzada em séries temporais

# Separando nossa variável de interesse.
X = walmart.drop(['Date','Weekly_Sales'], axis=1)
y = walmart['Weekly_Sales']

# Utilize a função para dividir os dados em treino e teste
X_training, X_test, y_training, y_test = train_test_split(X, y,
                                                        test_size=0.25,
                                                        shuffle=False)

print("Conjunto de treino X:", X_training.shape)
print("Conjunto de treino y:", y_training.shape)
print("Conjunto de teste  X:", X_test.shape)
print("Conjunto de teste  y:", y_test.shape)

# utilizando uma janela expansiva com n_splits=10 para a validação cruzada
cv = TimeSeriesSplit(n_splits=10)

Conjunto de treino X: (211838, 16)
Conjunto de treino y: (211838,)
Conjunto de teste  X: (70613, 16)
Conjunto de teste  y: (70613,)


## Modelagem

Agora iremos partir para a segunda etapa do nosso projeto, a modelagem.


#### Random Forest

Vamos iniciar treinando um modelo de *random forest* com validação cruzada e *grid search*.

O *grid search* deve variar, pelo menos, os parâmetros `n_estimators` e `max_depth`, mas você pode incluir outros.

Após o treino, criaremos o *dataframe* `cv_results` com os resultados de cada iteração e o dicionário `cv_best_params` com os valores da melhor combinação de parâmetros.

In [ ]:
# Random Forest com Validação cruzada e Grid Search para Séries temporais

from sklearn.metrics import (explained_variance_score,
                             mean_absolute_error,
                             mean_squared_error,
                             mean_squared_log_error,
                             r2_score)
from sklearn.ensemble import RandomForestRegressor # Modelo para aplicar de Random Forest
from sklearn.model_selection import GridSearchCV  # Modelo para aplicar Grid Search e Cross-Validation

# Defina os valores possíveis para os parâmetros a serem testados
params = {'n_estimators': [100],
          'max_depth': [10]}

#params = {'n_estimators': [50, 100],    testados
#          'max_depth': [5, 10]}

# Crie o modelo de random forest
rf_model_cv_gs = RandomForestRegressor()

# Crie o objeto para grid search
grid_search = GridSearchCV(rf_model_cv_gs,
                           param_grid=params,
                           return_train_score=True,
                           scoring='r2',
                           cv=cv)

# Treine o modelo com Grid Search
grid_search.fit(X_training, y_training)

# Estruture em um DataFrame os resultados da validação cruzada (cv_results)
cv_results = pd.DataFrame(grid_search.cv_results_)

# Armazene a melhor combinação de hiperparâmetros
cv_best_params = grid_search.best_params_
print('\n Best hyperparameters:')
print(cv_best_params)


 Best hyperparameters:
{'max_depth': 10, 'n_estimators': 100}


Agora vamos treinar o modelo final de *random forest* com os melhores parâmetros obtidos no *grid search* do exercício anterior.

In [ ]:
# Imprimindo o score médio nos conjuntos de treino
print("Pontuação média nos dados de treino: {:.3f} +/- {:.3f}".format(cv_results[cv_results.rank_test_score == 1].mean_train_score.values[0],
                                                                     cv_results[cv_results.rank_test_score == 1].std_train_score.values[0]))
# Imprimindo o score médio nos conjuntos de validação
print("Pontuação média nos dados de validação: {:.3f} +/- {:.3f}".format(cv_results[cv_results.rank_test_score == 1].mean_test_score.values[0],
                                                                     cv_results[cv_results.rank_test_score == 1].std_test_score.values[0]))

# Configure o modelo com a melhor combinação de hiperparâmetros
rf_model_cv_gs.set_params(n_estimators = cv_best_params['n_estimators'],
                        max_depth = cv_best_params['max_depth'])

# Treine um modelo com a melhor combinação de hiperparâmetros
rf_model_cv_gs.fit(X_training, y_training)
best_model_params = rf_model_cv_gs.get_params()

Pontuação média nos dados de treino: 0.902 +/- 0.022
Pontuação média nos dados de validação: 0.831 +/- 0.087


Por fim, vamos calcular o RMSE do modelo de *random forest* final na base de teste.

In [ ]:
y_test_pred_rf = rf_model_cv_gs.predict(X_test)
rmse_test_rf = math.sqrt(mean_squared_error(y_test, y_test_pred_rf))
print("RMSE do modelo no conjunto de teste:", rmse_test_rf)

RMSE do modelo no conjunto de teste: 7123.066337336159


Vamos desenhar gráficos para comparar as previsões que com os dados reais.

Obs.: Novamente, basta deslizar a barra para explorar diferentes séries temporais.

In [ ]:
# Plotando a série temporal dos dados previstos para uma dada loja e departamento

from IPython.display import display
from ipywidgets import interact, IntSlider

# juntando a previsão e os dados originais no mesmo dataset para a plotagem
training = pd.concat([X_training[['Store','Dept']], y_training], axis=1)
training['Predicted'] = rf_model_cv_gs.predict(X_training)
training = training.join(walmart['Date'])

test = pd.concat([X_test[['Store','Dept']], y_test], axis=1)
test['Predicted'] = rf_model_cv_gs.predict(X_test)
test = test.join(walmart['Date'])

# função de plot interativo, dada uma loja e um departamento
def plot_prediction(store, dept):
    # Filtrando a loja e o departamento para o plot
    plot_train = training[(training['Store']==store) & (training['Dept']==dept)]
    plot_test  = test[(test['Store']==store) & (test['Dept']==dept)]

    # Plotando no tempo os dados de treino e teste originais e previstos
    fig, ax = plt.subplots(figsize=(10,3))
    ax.plot(plot_train['Date'], plot_train['Weekly_Sales'])
    ax.plot(plot_train['Date'], plot_train['Predicted'], color='yellow')
    ax.plot(plot_test['Date'], plot_test['Weekly_Sales'], color='blue')
    ax.plot(plot_test['Date'], plot_test['Predicted'], color='orange')
    plt.show()

# Criando o Slider interativo
_ = interact(plot_prediction,
             store=IntSlider(min=walmart.Store.min(), max=walmart.Store.max(), step=1, value=20),
             dept =IntSlider(min=walmart.Dept.min(), max=walmart.Dept.max(), step=1, value=72)
             )

interactive(children=(IntSlider(value=20, description='store', max=45, min=1), IntSlider(value=72, description…